<a href="https://colab.research.google.com/github/ss1111119/Indigenous-Language-e-Paradise/blob/main/%E6%AD%8C%E8%AC%A0%E7%AF%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 歌謠篇
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 定義方言與語言組的對應關係字典
language_groups = {
    1: "南勢阿美語",
    2: "秀姑巒阿美語",
    3: "海岸阿美語",
    4: "馬蘭阿美語",
    5: "恆春阿美語",
    6: "賽考利克泰雅語",
    7: "澤敖利泰雅語",
    8: "汶水泰雅語",
    9: "萬大泰雅語",
    10: "四季泰雅語",
    11: "宜蘭澤敖利泰雅語",
    13: "賽夏語",
    14: "邵語",
    15: "都達賽德克語",
    16: "德固達雅賽德克語",
    17: "德鹿谷賽德克語",
    18: "卓群布農語",
    19: "卡群布農語",
    20: "丹群布農語",
    21: "巒群布農語",
    22: "郡群布農語",
    23: "東排灣語",
    24: "北排灣語",
    25: "中排灣語",
    26: "南排灣語",
    27: "東魯凱語",
    28: "霧台魯凱語",
    29: "大武魯凱語",
    30: "多納魯凱語",
    31: "茂林魯凱語",
    32: "萬山魯凱語",
    33: "太魯閣語",
    34: "噶瑪蘭語",
    35: "鄒語",
    38: "南王卑南語",
    39: "知本卑南語",
    40: "西群卑南語",
    41: "建和卑南語",
    42: "雅美語",
    43: "撒奇萊雅語",
    36: "卡那卡那富語",
    37: "拉阿魯哇語"
}

def get_dialect_info():
    dialect_info = {}

    # 遍歷每個方言的目標URL
    for dialect_id, data in all_target_urls.items():
        urls_for_dialect = data['urls']

        # 如果存在目標URL，則請求網頁內容並提取信息
        if urls_for_dialect[0]:
            dialect_info[dialect_id] = {
                "language_group": data['language_group'],
                "sentences": []  # 存儲族語句子和中文翻譯
            }

            for url in urls_for_dialect:
                response = requests.get(url)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    title = soup.find('title').text.strip() if soup.find('title') else '未知標題'
                    ab_sentence_elements = soup.find_all('div', class_='read-sentence Ab')

                    for ab_element in ab_sentence_elements:
                        ab_sentence = ' '.join(word.text.strip() for word in ab_element.find_all('div', class_='word'))
                        data_value = ab_element.find('button', class_='read-chinese-btn')['data-value']
                        ch_translation_block = soup.find('div', class_='read-sentence Ch', attrs={'data-value': data_value})
                        ch_translation = ch_translation_block.text.strip() if ch_translation_block else "未找到中文翻譯"
                        dialect_info[dialect_id]["sentences"].append({"title": title, "ab_sentence": ab_sentence, "ch_translation": ch_translation})
                else:
                    print(f"請求失敗，狀態碼: {response.status_code}")

    return dialect_info

# 獲取所有方言對應的目標URL
all_target_urls = get_target_urls()

# 獲取方言信息（包括族語句子和中文翻譯）
dialect_info = get_dialect_info()

# 打印方言信息
for dialect_id, data in dialect_info.items():
    print(f"方言ID {dialect_id}，語言組: {data['language_group']}，的族語句子和中文翻譯:")
    dialect_data = []  # 存儲當前方言的數據
    prev_title = None

    for sentence_data in data["sentences"]:
        title = sentence_data['title']
        ab_sentence = sentence_data['ab_sentence']
        ch_translation = sentence_data['ch_translation']

        if title != prev_title:
            prev_title = title

        # 將標題、族語句子和中文翻譯合併成一個列表
        dialect_data.append((title, ab_sentence, ch_translation))

    # 建立DataFrame
    df = pd.DataFrame(dialect_data, columns=['標題', '族語', '中文'])

    # 將單個方言的數據存儲為Excel文件
    filename = f"{dialect_id}_{language_groups.get(dialect_id, '未知語言')}.xlsx"
    df.to_excel(filename, index=False)
    print(f"{language_groups.get(dialect_id, '未知語言')}的數據已保存至 {filename}")
